# Notebook de Benchmarking

In [ ]:
# ------------------------------------------------------------
# Env configuration
# ------------------------------------------------------------
import sys
import os
from dotenv import load_dotenv

import pandas as pd

# Base path of the project (goes up one level from the current directory)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Check if `src` is in sys.path, add it if not
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"'src' added to sys.path: {SRC_PATH}")

# Ensure the `loaders` folder exists within `src`
loaders_path = os.path.join(SRC_PATH, "loaders")
if not os.path.exists(loaders_path):
    raise FileNotFoundError(f"Folder 'loaders' not found in {SRC_PATH}. Check the project structure.")

# Import required modules from the project
from retrievers.retriever import Retriever
from langchain_core.documents import Document
import evaluation.evaluation_ragas as eval
from qdrant_client import QdrantClient


'src' agregado a sys.path: g:\Mi unidad\EnorChile\GIT\practicos-rag\src


In [ ]:
# ------------------------------------------------------------
#  Qdrant Connection
# ------------------------------------------------------------

# Load environment variables from the .env file
load_dotenv()

# Read variables from .env
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
INDEX_NAME = "embeddings-version-2"

# Validate that the variables were loaded correctly
if not QDRANT_URL or not QDRANT_API_KEY:
    raise ValueError("The variables QDRANT_URL or QDRANT_API_KEY are not properly configured in the .env file")

print("Connecting to Qdrant...")
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
print("Connection to Qdrant successfully established.")

Conectando a Qdrant...
Conexión a Qdrant establecida con éxito.


In [5]:
# ------------------------------------------------------------
# Generate Embedding
# ------------------------------------------------------------

from openai import OpenAI  # Import the OpenAI client library

# Initialize the OpenAI client using the API key from environment variables
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_query_embedding(query):
    """
    Generate an embedding for a given query using the 'text-embedding-ada-002' model.

    Args:
        query (str): The input text query for which the embedding is generated.

    Returns:
        list: A list representing the embedding vector of the input query.
    """
    # Request the embedding for the input query from OpenAI
    response = client.embeddings.create(
        model="text-embedding-ada-002",  # Specify the model to use for embeddings
        input=query  # The input query text
    )
    # Extract the embedding vector from the API response
    embedding = response.data[0].embedding
    
    return embedding  # Return the embedding vector

In [6]:
# ------------------------------------------------------------
# Qdrant Search 
# ------------------------------------------------------------

def search_qdrant(qdrant_client, index_name, query_vector, limit=5):
    """
    Perform a search in Qdrant to find the most similar points.

    Parameters:
        - qdrant_client: Qdrant client instance.
        - index_name: Name of the collection to query.
        - query_vector: The query vector for the search.
        - limit: Number of desired results (default is 5).

    Returns:
        - List of found results, including vectors and payloads.
    """
    # Query the Qdrant collection for points similar to the query vector
    results = qdrant_client.query_points(
                    collection_name=index_name,  # Name of the collection
                    query=query_vector,          # Query vector for similarity search
                    limit=limit,                 # Limit the number of results
                    using="default",             # Use the default query method
                    with_payload=True,           # Include payloads in the results
                    with_vectors=True,           # Include vectors in the results
                )
    return results.points  # Return the list of matching points


In [7]:
# ------------------------------------------------------------
# Combine embedding and vector searching
# ------------------------------------------------------------

def embed_and_search(query, collection, limit=5):
    # Generate embedding for the query
    try:
        query_embedding = get_query_embedding(query)
        if not isinstance(query_embedding, list) or len(query_embedding) != 1536:
            raise ValueError("The query embedding does not have the expected format (list with 1536 dimensions).")

        # Validate the format of the generated embedding
        
    except Exception as e:
        print(f"Error generating the query embedding: {e}")
        raise

    index_name = collection  # Name of the collection in Qdrant

    try:
        results = search_qdrant(qdrant_client, index_name, query_embedding, limit=limit)
    except Exception as e:
        print(f"Error performing the search in Qdrant: {e}")
        results = []

    return results

In [8]:
# ------------------------------------------------------------
# Evaluation Routine
# ------------------------------------------------------------

eval_df = pd.read_excel('evaluacion.xlsx')
coleccion = "openai-750-overlap"
limit = 30

# Initialize an empty list to collect results for each iteration
eval_results = []

for _, row in eval_df.iterrows():
    query = row.iloc[0]  # Assuming the query is in the first column
    ground_truth = row.iloc[1]
    print(f'Query {_}/{len(eval_df)}: {query}')
    
    # Perform embedding and search
    results = embed_and_search(query, coleccion, limit=limit)
    chunks_basic = [point.payload['content'] for point in results]
    chunks_advanced = Retriever().summarization(chunks_basic)['docs']
    
    list_documents = []

    for doc in chunks_basic:
        document = Document(
            page_content=doc,
            metadata={}
        )
        list_documents.append(document)
    answer_basic, answer_adv = Retriever().process_query(list_documents=list_documents, enhanced=True, question=query,context=chunks_basic)
    
    # Evaluate RAGAS for basic and advanced retrieval
    ragas_basic = eval.evaluate_ragas(query, answer_basic, chunks_basic, ground_truth)
    ragas_basic['retrieval'] = "Basic"
    
    ragas_adv = eval.evaluate_ragas(query, answer_adv, chunks_advanced, ground_truth)
    ragas_adv['retrieval'] = "Advanced"
    
    # Append results to the list
    eval_results.append(ragas_basic)
    eval_results.append(ragas_adv)

# Consolidate all results into a single DataFrame
df_eval = pd.concat(eval_results, ignore_index=True)

Query 0/10: ¿Qué se entiende por cerveza?


g:\Mi unidad\EnorChile\GIT\practicos-rag\src\retrievers\retriever.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name="gpt-3.5-turbo")
g:\Mi unidad\EnorChile\GIT\practicos-rag\src\retrievers\retriever.py:62: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  map_chain = LLMChain(llm=self.llm, prompt=map_prompt)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Query 1/10: ¿Cómo se clasifican las cervezas?


...

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Query 9/10: ¿Qué antioxidantes o sinergistas pueden agregarse a los aceites y grasas vegetales comestibles?


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
df_eval.to_csv(f'evaluacion_{coleccion}.csv', index=False)